In [1]:
from pyscf import gto, scf, ao2mo, fci
from GeneralisedWick import *
from scipy.optimize import fsolve

In [2]:
def commutator(operator1, operator2):
    return operator1 * operator2 + (-1) * operator2 * operator1

In [3]:
def BCHSimilarityTransform(H, T, order):
    result = H
    for k in range(order):
        nestedCommutator = H
        for i in range(k + 1):
            nestedCommutator = commutator(nestedCommutator, T)
        result += (1 / factorial(k + 1)) * nestedCommutator
    return result

In [4]:
def projectionManifold(excitationLevel):
    return Tensor('\Phi', ['h'] * excitationLevel, ['p'] * excitationLevel)

In [5]:
fockTensor = Tensor("f", ['g'], ['g'])
h1Tensor = Tensor("h", ['g'], ['g'])
h2Tensor = Tensor("v", ['g', 'g'], ['g', 'g'])
t2Tensor = Tensor("{t_{2}}", ['p', 'p'], ['h', 'h'])

In [6]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'Be 0 0 0'
#mol.basis = 'sto-3g'
#mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = '6-31g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

cisolver = fci.FCI(mol, mf.mo_coeff)

Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1] * Nocc + [0] * (Norbs - Nocc)

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff, compact=False)

In [7]:
from pyscf import cc, ccn

In [8]:
trueCCD = cc.CCSD(mf)

In [9]:
trueCCD.update_amps

<bound method update_amps of <pyscf.cc.uccsd.UCCSD object at 0x7f5ad4ea0588>>

In [10]:
old_update_amps = trueCCD.update_amps
def update_amps(t1, t2, eris):
    t1, t2 = old_update_amps(t1, t2, eris)
#    print(t1)
    return (np.zeros_like(t1[0]), np.zeros_like(t1[1])), t2
trueCCD.update_amps = update_amps

In [11]:
trueCCD.kernel()

(-0.046469771227442426,
 (array([[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]]),
  array([[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]])),
 (array([[[[ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ]],
  

In [12]:
trueCCD.update_amps

<function __main__.update_amps(t1, t2, eris)>

In [13]:
trueCCD

In [14]:
trueCCD.kernel()

(-0.046469771227442426,
 (array([[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]]),
  array([[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]])),
 (array([[[[ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ]],
  

In [15]:
trueAmplitudeArray = trueCCD.kernel()[2][1].swapaxes(0,2).swapaxes(1,3)

In [16]:
trueAmplitudeArray

array([[[[ 7.25036639e-05, -1.22582208e-03],
         [-1.22582208e-03, -1.71775938e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-8.32519695e-04, -1.22019758e-03],
         [-2.35857990e-03, -5.57834681e-02]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 7.25036639e-05, -1.22582208e-03],
         [-1.22582208e-03, -1.71775938e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.000

In [17]:
#t2Tensor.array

In [18]:
def get2bodyHamiltonianArray(mf_):
    eri = ao2mo.kernel(mf_.mol, mf_.mo_coeff)
    Norbs_ = mf_.mol.nao
    hamiltonian2BodyArray = np.zeros((Norbs_,Norbs_,Norbs_,Norbs_))
    for p in range(Norbs_):
        for q in range(Norbs_):
            for r in range(p + 1):
                for s in range(q + 1):
                    x = int(p + Norbs_ * r - 0.5 * r * (r + 1))
                    y = int(q + Norbs_ * s - 0.5 * s * (s + 1))
                    if p == r and q == s:
                        hamiltonian2BodyArray[p,q,r,s] += eri[x, y]
                    else:
                        hamiltonian2BodyArray[p,q,r,s] += eri[x, y]
                        hamiltonian2BodyArray[r,s,p,q] += np.conjugate(eri[x, y])
    return hamiltonian2BodyArray

In [19]:
h1Tensor.array = h1
h2Tensor.array = eri.reshape((Norbs, Norbs, Norbs, Norbs)).swapaxes(2,3).swapaxes(1,2)
t2Tensor.getShape(vacuum)

In [20]:
#t2Tensor.array[0, 0, 0, 0] = -0.275333314977893 * 0.5
t2Tensor.array = np.zeros(t2Tensor.array.shape)
#t2Tensor.array = trueAmplitudeArray

In [21]:
h1

array([[-7.94050428,  0.20663739,  0.        ,  0.        ,  0.        ,
         0.17965963,  0.        ,  0.        ,  0.        ],
       [ 0.20663739, -1.61807386,  0.        ,  0.        ,  0.        ,
        -0.43217769,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.19628378,  0.        ,  0.        ,
         0.        , -0.48516619,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -1.19628378,  0.        ,
         0.        ,  0.        ,  0.        , -0.48516619],
       [ 0.        ,  0.        ,  0.        ,  0.        , -1.19628378,
         0.        ,  0.        , -0.48516619,  0.        ],
       [ 0.17965963, -0.43217769,  0.        ,  0.        ,  0.        ,
        -0.95456897,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.48516619,  0.        ,  0.        ,
         0.        , -1.3410308 ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0

In [22]:
mf.mo_energy

NPArrayWithTag([-4.7068905 , -0.30129539,  0.08243534,  0.08243534,
                 0.08243534,  0.43975431,  0.46493101,  0.46493101,
                 0.46493101])

In [23]:
eri

array([[ 2.27130649, -0.19996916,  0.        , ...,  0.        ,
         0.        ,  0.58302395],
       [-0.19996916,  0.0283882 ,  0.        , ...,  0.        ,
         0.        , -0.0096241 ],
       [ 0.        ,  0.        ,  0.00330086, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.01877945,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.01877945,  0.        ],
       [ 0.58302395, -0.0096241 ,  0.        , ...,  0.        ,
         0.        ,  0.39402566]])

In [24]:
h2Tensor.array

array([[[[ 2.27130649e+00, -1.99969155e-01,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [-1.99969155e-01,  2.83882044e-02,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  3.30086167e-03, ...,
           9.35905544e-03,  0.00000000e+00,  0.00000000e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  9.35905544e-03, ...,
           2.69900982e-02,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  2.69900982e-02,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  2.69900982e-02]],

        [[-1.99969155e-01,  4.95347745e-01,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 2.83882044e-02, -6.66823217e-03,  0.00000000e+00, ...,
           0.00000000e

In [25]:
#fock = mf.get_fock(h1e=h1, vhf=eri)

In [26]:
#print(fock)
#print(np.linalg.eig(fock))

In [27]:
eri.reshape((Norbs, Norbs, Norbs, Norbs)).swapaxes(2,3).swapaxes(1,2)

array([[[[ 2.27130649e+00, -1.99969155e-01,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [-1.99969155e-01,  2.83882044e-02,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  3.30086167e-03, ...,
           9.35905544e-03,  0.00000000e+00,  0.00000000e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  9.35905544e-03, ...,
           2.69900982e-02,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  2.69900982e-02,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  2.69900982e-02]],

        [[-1.99969155e-01,  4.95347745e-01,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 2.83882044e-02, -6.66823217e-03,  0.00000000e+00, ...,
           0.00000000e

In [28]:
fock = h1
for p in range(Norbs):
    for q in range(Norbs):
        fock[p,q] += sum([2 * h2Tensor.array[p,i,q,i] - h2Tensor.array[p,i,i,q] for i in range(Nocc)])

In [29]:
fock

array([[-4.70689051e+00,  5.31283906e-11,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  3.21496441e-11,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 5.31285016e-11, -3.01295386e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  8.48783932e-10,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  8.24353414e-02,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -6.09065243e-10,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         8.24353414e-02,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -6.09065409e-10],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  8.24353414e-02,  0.00000000e+00,
         0.00000000e+00, -6.09065409e-10,  0.00000000e+00],
       [ 3.21496163e-11,  8.48783821e-10,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  4.397543

In [30]:
fockTensor.array = fock

In [31]:
np.linalg.eig(fock)

(array([-4.70689051, -0.30129539,  0.43975431,  0.08243534,  0.46493101,
         0.08243534,  0.46493101,  0.46493101,  0.08243534]),
 array([[-1.00000000e+00,  1.20592994e-11,  6.24671903e-12,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.20593246e-11,  1.00000000e+00,  1.14538049e-09,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00, -1.00000000e+00,
          1.59234547e-09,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -1.00000000e+00,  1.59234591e-09,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e

In [32]:
mf.mo_energy

NPArrayWithTag([-4.7068905 , -0.30129539,  0.08243534,  0.08243534,
                 0.08243534,  0.43975431,  0.46493101,  0.46493101,
                 0.46493101])

Hard code spin-free equations from Bartlett & Shavitt Chapter 12

In [33]:
fockDiagrams = fockTensor.getAllDiagrams(vacuum)
h2Diagrams = h2Tensor.getAllDiagrams(vacuum)

In [34]:
print(*fockDiagrams)

f_{h}^{h} f_{h}^{p} f_{p}^{h} f_{p}^{p}


In [35]:
print([fockDiagram.array.shape for fockDiagram in fockDiagrams])

[(2, 2), (2, 7), (7, 2), (7, 7)]


In [36]:
print(*h2Diagrams)

v_{hh}^{hh} v_{hh}^{hp} v_{hh}^{ph} v_{hh}^{pp} v_{hp}^{hh} v_{hp}^{hp} v_{hp}^{ph} v_{hp}^{pp} v_{ph}^{hh} v_{ph}^{hp} v_{ph}^{ph} v_{ph}^{pp} v_{pp}^{hh} v_{pp}^{hp} v_{pp}^{ph} v_{pp}^{pp}


In [37]:
print([h2Diagram.array.shape for h2Diagram in h2Diagrams])

[(2, 2, 2, 2), (2, 2, 2, 7), (2, 2, 7, 2), (2, 2, 7, 7), (2, 7, 2, 2), (2, 7, 2, 7), (2, 7, 7, 2), (2, 7, 7, 7), (7, 2, 2, 2), (7, 2, 2, 7), (7, 2, 7, 2), (7, 2, 7, 7), (7, 7, 2, 2), (7, 7, 2, 7), (7, 7, 7, 2), (7, 7, 7, 7)]


In [38]:
I = Index("I", True)
J = Index("J", True)
K = Index("K", True)
L = Index("L", True)
A = Index("A", False)
B = Index("B", False)
C = Index("C", False)
D = Index("D", False)

In [39]:
D1 = TensorSum([TensorProduct([h2Diagrams[12]], vertexList=[Vertex(h2Diagrams[12], [A,B], [I,J])])])

In [40]:
D2a = TensorSum([TensorProduct([fockDiagrams[3], t2Tensor], vertexList=[Vertex(fockDiagrams[3], [B], [C]), Vertex(t2Tensor, [A,C], [I,J])]), TensorProduct([fockDiagrams[3], t2Tensor], vertexList=[Vertex(fockDiagrams[3], [A], [C]), Vertex(t2Tensor, [B,C], [J,I])])])

In [41]:
D2b = TensorSum([TensorProduct([fockDiagrams[0], t2Tensor], vertexList=[Vertex(fockDiagrams[0], [K], [J]), Vertex(t2Tensor, [A,B], [I,K])]), TensorProduct([fockDiagrams[0], t2Tensor], vertexList=[Vertex(fockDiagrams[0], [K], [I]), Vertex(t2Tensor, [B,A], [J,K])])])

In [42]:
D2c = TensorSum([TensorProduct([h2Diagrams[15], t2Tensor], vertexList=[Vertex(h2Diagrams[15], [A,B], [C,D]), Vertex(t2Tensor, [C,D], [I,J])]), TensorProduct([h2Diagrams[15], t2Tensor], vertexList=[Vertex(h2Diagrams[15], [B,A], [C,D]), Vertex(t2Tensor, [C,D], [J,I])])])

In [43]:
D2d = TensorSum([TensorProduct([h2Diagrams[0], t2Tensor], vertexList=[Vertex(h2Diagrams[0], [K,L], [I,J]), Vertex(t2Tensor, [A,B], [K,L])]), TensorProduct([h2Diagrams[0], t2Tensor], vertexList=[Vertex(h2Diagrams[0], [K,L], [J,I]), Vertex(t2Tensor, [B,A], [K,L])])])

In [44]:
D2e = TensorSum([TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,B], [C,J]), Vertex(t2Tensor, [A,C], [I,K])]), TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,A], [C,I]), Vertex(t2Tensor, [B,C], [J,K])])])

In [45]:
D2ex1 = TensorSum([TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,B], [C,J]), Vertex(t2Tensor, [A,C], [K,I])]), TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,A], [C,I]), Vertex(t2Tensor, [B,C], [K,J])])])

In [46]:
D2ex2 = TensorSum([TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,B], [I,C]), Vertex(t2Tensor, [A,C], [K,J])]), TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,A], [J,C]), Vertex(t2Tensor, [B,C], [K,I])])])

In [47]:
D2ex3 = TensorSum([TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,B], [J,C]), Vertex(t2Tensor, [A,C], [I,K])]), TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,A], [I,C]), Vertex(t2Tensor, [B,C], [J,K])])])

In [48]:
D3a = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [I,J]), Vertex(t2Tensor, [A,B], [K,L])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [J,I]), Vertex(t2Tensor, [B,A], [K,L])])])

In [49]:
D3b = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,C], [I,K]), Vertex(t2Tensor, [D,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,C], [J,K]), Vertex(t2Tensor, [D,A], [L,I])])])

In [50]:
D3bx1 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,C], [I,K]), Vertex(t2Tensor, [D,B], [J,L])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,C], [J,K]), Vertex(t2Tensor, [D,A], [I,L])])])

In [51]:
D3bx2 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,A], [I,K]), Vertex(t2Tensor, [B,D], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,B], [J,K]), Vertex(t2Tensor, [A,D], [L,I])])])

In [52]:
D3bx3 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,D], [I,K]), Vertex(t2Tensor, [C,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,D], [J,K]), Vertex(t2Tensor, [C,A], [L,I])])])

In [53]:
D3bx4 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,D], [K,I]), Vertex(t2Tensor, [C,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,D], [K,J]), Vertex(t2Tensor, [C,A], [L,I])])])

In [54]:
D3bx5 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,B], [I,L]), Vertex(t2Tensor, [A,D], [K,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,A], [J,L]), Vertex(t2Tensor, [B,D], [K,I])])])

In [55]:
D3c = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [K,I]), Vertex(t2Tensor, [A,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [K,J]), Vertex(t2Tensor, [B,A], [L,I])])])

In [56]:
D3cx = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [I,K]), Vertex(t2Tensor, [A,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [J,K]), Vertex(t2Tensor, [B,A], [L,I])])])

In [57]:
D3d = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,A], [K,L]), Vertex(t2Tensor, [D,B], [I,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,B], [K,L]), Vertex(t2Tensor, [D,A], [J,I])])])

In [58]:
D3dx = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,C], [K,L]), Vertex(t2Tensor, [D,B], [I,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,C], [K,L]), Vertex(t2Tensor, [D,A], [J,I])])])

In [59]:
amplitudeDiagrams = [D1, D2a, D2b, D2c, D2d, D2e, D2ex1, D2ex2, D2ex3, D3a, D3b, D3bx1, D3bx2, D3bx3, D3bx4, D3bx5, D3c, D3cx, D3d, D3dx]
amplitudeFactors = [1., 1., -1., 0.5, 0.5, 2., -1., -1., -1., 0.5, 2., -2., 0.5, -1., 1., 0.5, -2., 1., -2., 1.]

In [60]:
print(D2a)

1.0 * f_{B}^{C}{t_{2}}_{AC}^{IJ}
 + 1.0 * f_{A}^{C}{t_{2}}_{BC}^{JI}


In [61]:
D2a.summandList[0].vertexList[1].tensor.array.shape

(7, 7, 2, 2)

In [62]:
contractTensorSum(TensorSum([D2a.summandList[0]])).shape

(7, 7, 2, 2)

In [63]:
def testContractedArray(tensorProduct_, targetLowerIndexList=[A,B], targetUpperIndexList=[I,J]):
    lowerIndexList = list(itertools.chain.from_iterable([vertex.lowerIndices for vertex in tensorProduct_.vertexList]))
    upperIndexList = list(itertools.chain.from_iterable([vertex.upperIndices for vertex in tensorProduct_.vertexList]))
    lowerIndexLetters = string.ascii_lowercase[:len(lowerIndexList)]
    upperIndexLetters = ''
    freeLowerIndexMask = np.ones(len(lowerIndexList))
    freeUpperIndexMask = np.ones(len(upperIndexList))
    nFreeUpperIndices = 0
    for uI, upperIndex in enumerate(upperIndexList):
        free = True
        for lI, lowerIndex in enumerate(lowerIndexList):
            if upperIndex == lowerIndex:
                #print(upperIndex, lowerIndex)
                upperIndexLetters += lowerIndexLetters[lI]
                freeLowerIndexMask[lI] = 0
                freeUpperIndexMask[uI] = 0
                free = False
        if free:
            upperIndexLetters += string.ascii_lowercase[len(lowerIndexList) + nFreeUpperIndices]
            nFreeUpperIndices += 1
    newLowerIndexList = [lowerIndex for lI, lowerIndex in enumerate(lowerIndexList) if freeLowerIndexMask[lI]]
    newUpperIndexList = [upperIndex for uI, upperIndex in enumerate(upperIndexList) if freeUpperIndexMask[uI]]
    summandZero = False
    if targetLowerIndexList == None and targetUpperIndexList == None:
        targetLowerIndexList = newLowerIndexList
        targetUpperIndexList = newUpperIndexList
        summandZero = True
    freeLowerIndexLetters = "".join([lowerIndexLetters[lowerIndexList.index(lowerIndex)] for lowerIndex in targetLowerIndexList])
    freeUpperIndexLetters = "".join([upperIndexLetters[upperIndexList.index(upperIndex)] for upperIndex in targetUpperIndexList])
    einsumSubstrings = []
    start = 0
    for vertex in tensorProduct_.vertexList:
        end = start + vertex.excitationRank
        einsumSubstring = lowerIndexLetters[start:end] + upperIndexLetters[start:end]
        einsumSubstrings.append(einsumSubstring)
        start = end
    einsumString = ",".join(einsumSubstrings)
    einsumString += '->' + freeLowerIndexLetters + freeUpperIndexLetters
    print(*lowerIndexList, *upperIndexList)
    print(*lowerIndexLetters, *upperIndexLetters)
    print(*newLowerIndexList, *newUpperIndexList)
    print(*freeLowerIndexLetters, *freeUpperIndexLetters)
    print(einsumString)
    contribution = tensorProduct_.prefactor * np.einsum(einsumString, *[vertex.tensor.array for vertex in tensorProduct_.vertexList])
    #print(*newLowerIndexList, *newUpperIndexList)
    #if summandZero:
    #    return contribution, newLowerIndexList, newUpperIndexList
    return contribution

In [64]:
t2Tensor.array.shape

(7, 7, 2, 2)

In [65]:
tCA = testContractedArray(D2a.summandList[0])
print(D2a.summandList[1])
print(tCA)
print(tCA.shape)
#print(*tCA[1])
#print(*tCA[2])

B A C C I J
a b c c d e
B A I J
b a d e
ac,bcde->bade
1.0 * f_{A}^{C}{t_{2}}_{BC}^{JI}
[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [

In [66]:
amplitudes = TensorSum([])
for d, diagram in enumerate(amplitudeDiagrams):
    for summand in diagram.summandList:
        amplitudes.summandList.append(copy(summand))
        amplitudes.summandList[-1].prefactor *= amplitudeFactors[d]
    print(diagram)
    print(contractTensorSum(diagram))
#    print(amplitudeFactors[d] * contractTensorSum(diagram))

1.0 * v_{AB}^{IJ}
[[[[ 0.00330086  0.00659818]
   [ 0.00659818  0.063035  ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00935906  0.00673648]
   [ 0.01561124  0.02848843]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]]


 [[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00330086  0.00659818]
   [ 0.00659818  0.063035  ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00935906  0.00673648]
   [ 0.01561124  0.02848843]]]


 [[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00330086  0

In [67]:
#t2Tensor.array[0, 0, 0, 0] = -0.38

In [68]:
h2Tensor.array = get2bodyHamiltonianArray(mf)

In [69]:
residualTerms = [amplitudeFactors[d] * contractTensorSum(diagram) for d, diagram in enumerate(amplitudeDiagrams)]

In [70]:
sum([testContractedArray(term) for term in amplitudes.summandList])

A B I J
a b c d
A B I J
a b c d
abcd->abcd
B A C C I J
a b c c d e
B A I J
b a d e
ac,bcde->bade
A B C C J I
a b c c d e
A B J I
a b e d
ac,bcde->abed
K A B J I K
a b c d e a
A B J I
b c e d
ad,bcea->bced
K B A I J K
a b c d e a
B A I J
c b d e
ad,bcea->cbde
A B C D C D I J
a b c d c d e f
A B I J
a b e f
abcd,cdef->abef
B A C D C D J I
a b c d c d e f
B A J I
b a f e
abcd,cdef->bafe
K L A B I J K L
a b c d e f a b
A B I J
c d e f
abef,cdab->cdef
K L B A J I K L
a b c d e f a b
B A J I
d c f e
abef,cdab->dcfe
K B A C C J I K
a b c d d e f a
B A J I
c b f e
abde,cdfa->cbfe
K A B C C I J K
a b c d d e f a
A B I J
b c e f
abde,cdfa->bcef
K B A C C J K I
a b c d d e a f
B A J I
c b f e
abde,cdaf->cbfe
K A B C C I K J
a b c d d e a f
A B I J
b c e f
abde,cdaf->bcef
K B A C I C K J
a b c d e d a f
B A I J
c b e f
abed,cdaf->cbef
K A B C J C K I
a b c d e d a f
A B J I
b c f e
abed,cdaf->bcfe
K B A C J C I K
a b c d e d f a
B A J I
c b f e
abed,cdfa->cbfe
K A B C I C J K
a b c d e d f a
A B I

array([[[[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00935906,  0.00673648],
         [ 0.01561124,  0.02848843]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [71]:
contractTensorSum(amplitudes)

array([[[[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00935906,  0.00673648],
         [ 0.01561124,  0.02848843]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [72]:
residualTerms

[array([[[[ 0.00330086,  0.00659818],
          [ 0.00659818,  0.063035  ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.00935906,  0.00673648],
          [ 0.01561124,  0.02848843]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]]],
 
 
        [[[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.00330086,  0.00659818],
          [ 0.00659818,  0.063035  ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.        ,  0.        ],
          [

In [73]:
constant = sum(residualTerms[:1])
linearFock = sum(residualTerms[1:3])
linearFluctuation = sum(residualTerms[3:9])
quadratic = sum(residualTerms[9:])

In [74]:
supposedLinearFluctuation = residualTerms[3] - residualTerms[4]
print(supposedLinearFluctuation)

[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0.

In [75]:
print(constant)
print(linearFock)
print(linearFluctuation)
print(quadratic)

[[[[ 0.00330086  0.00659818]
   [ 0.00659818  0.063035  ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00935906  0.00673648]
   [ 0.01561124  0.02848843]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]]


 [[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00330086  0.00659818]
   [ 0.00659818  0.063035  ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00935906  0.00673648]
   [ 0.01561124  0.02848843]]]


 [[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00330086  0.00659818]
   [ 0.

In [76]:
constant + linearFock + supposedLinearFluctuation + quadratic

array([[[[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00935906,  0.00673648],
         [ 0.01561124,  0.02848843]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [77]:
sum(residualTerms)

array([[[[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00935906,  0.00673648],
         [ 0.01561124,  0.02848843]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [78]:
sum(residualTerms[9:])

array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0

In [79]:
sum(residualTerms[:5]) + sum(residualTerms[9:])

array([[[[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00935906,  0.00673648],
         [ 0.01561124,  0.02848843]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [80]:
Ej = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [A,B]), Vertex(t2Tensor, [A,B], [I,J])])])
Ek = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [B,A]), Vertex(t2Tensor, [A,B], [I,J])])])

In [81]:
ECCD = 2 * contractTensorSum(Ej) - contractTensorSum(Ek)
print(ECCD)

0.0


In [82]:
def getCCDEnergy(doublesTensor):
    Ej = TensorSum([TensorProduct([h2Diagrams[3], doublesTensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [A,B]), Vertex(doublesTensor, [A,B], [I,J])])])
    Ek = TensorSum([TensorProduct([h2Diagrams[3], doublesTensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [B,A]), Vertex(doublesTensor, [A,B], [I,J])])])
    return 2 * contractTensorSum(Ej) - contractTensorSum(Ek)

In [83]:
t2Tensor.array * ECCD

array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0

In [84]:
print(Ej)
print(Ek)

1.0 * v_{IJ}^{AB}{t_{2}}_{AB}^{IJ}
1.0 * v_{IJ}^{BA}{t_{2}}_{AB}^{IJ}


Coupled cluster

In [85]:
print(t2Tensor.array.reshape((-1)))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [86]:
residualTensor = Tensor("R", ['p', 'p'], ['h', 'h'])

In [87]:
residualTensor.array = contractTensorSum(amplitudes)

In [88]:
print(residualTensor.array.reshape((-1)))

[ 0.00330086  0.00659818  0.00659818  0.063035    0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.00935906  0.00673648
  0.01561124  0.02848843  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.00330086  0.00659818  0.00659818  0.063035
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.00935906  0.00673648
  0.01561124  0.02848843  0.          0.          0.          0.
  0.          0.          0.          0.          0.00330086  0.00659818
  0.00659818  0.063035    0.          0.          0.          0.
  0.          0.          0.          0.          0.00935906  0.00673648
  0.01561124  0.02848843  0.          0.          0.          0.
  0.          0.          0.          0.          0.

In [89]:
print(t2Tensor.array)

[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0.

In [90]:
def doublesAmplitudesToVector(doublesAmplitudeTensor):
    vec = []
#    for i in range()
    return vec

In [91]:
def testAmplitudes(amplitudeVector):
    t2Tensor.array = amplitudeVector.reshape(t2Tensor.array.shape)
    residualTensor.array = contractTensorSum(amplitudes)
    return residualTensor.array.reshape((-1))

In [92]:
def getDoublesMP2Amplitudes(doublesTensor):
    mp2Amplitudes = np.zeros_like(doublesTensor.array)
    for i in range(mp2Amplitudes.shape[0]):
        for j in range(mp2Amplitudes.shape[1]):
            for k in range(mp2Amplitudes.shape[2]):
                for l in range(mp2Amplitudes.shape[3]):
#                    mp2Amplitudes[i,j,k,l] = -(2 - (i==j)) * h2Diagrams[12].array[i,j,k,l] / (fock[i + Nocc, i + Nocc] + fock[j + Nocc, j + Nocc] - fock[k, k] - fock[l, l])
                    mp2Amplitudes[i,j,k,l] = -h2Diagrams[12].array[i,j,k,l] / (fock[i + Nocc, i + Nocc] + fock[j + Nocc, j + Nocc] - fock[k, k] - fock[l, l])
    return mp2Amplitudes

In [93]:
def iterateDoublesAmplitudes(doublesTensor, residual):
    amplitudes = doublesTensor.array
    for i in range(amplitudes.shape[0]):
        for j in range(amplitudes.shape[1]):
            for k in range(amplitudes.shape[2]):
                for l in range(amplitudes.shape[3]):
#                    mp2Amplitudes[i,j,k,l] = (2 - (i==j)) * residual.array[i,j,k,l] / (fock[i + Nocc, i + Nocc] + fock[j + Nocc, j + Nocc] - fock[k, k] - fock[l, l])
                    amplitudes[i,j,k,l] -= residual.array[i,j,k,l] / (fock[i + Nocc, i + Nocc] + fock[j + Nocc, j + Nocc] - fock[k, k] - fock[l, l])
    return amplitudes

In [94]:
iterateDoublesAmplitudes(t2Tensor, h2Diagrams[12])

array([[[[-0.00034461, -0.00127549],
         [-0.00127549, -0.08213441]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00093956, -0.00121257],
         [-0.00281002, -0.02477347]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00034461, -0.00127549],
         [-0.00127549, -0.08213441]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [95]:
t2Tensor.array = np.zeros_like(t2Tensor.array)

In [96]:
Energy = getCCDEnergy(t2Tensor)

In [97]:
print(t2Tensor.array)

[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0.

In [98]:
print(getDoublesMP2Amplitudes(t2Tensor))

[[[[-0.00034461 -0.00127549]
   [-0.00127549 -0.08213441]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.00093956 -0.00121257]
   [-0.00281002 -0.02477347]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]]


 [[[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.00034461 -0.00127549]
   [-0.00127549 -0.08213441]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.00093956 -0.00121257]
   [-0.00281002 -0.02477347]]]


 [[[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.00034461 -0.00127549]
   [-0.

In [99]:
def convergeDoublesAmplitudes():
    t2Tensor.array = np.zeros_like(t2Tensor.array)
#    t2Tensor.array = trueAmplitudeArray
    Energy = getCCDEnergy(t2Tensor)
    residualTensor.array = h2Diagrams[12].array
    while True:
        print(Energy)
        t2Tensor.array = iterateDoublesAmplitudes(t2Tensor, residualTensor)
        residualTensor.array = contractTensorSum(amplitudes)
        Energy = getCCDEnergy(t2Tensor)
#        newEnergy = getCCDEnergy(t2Tensor)
#        if abs(newEnergy - Energy) < 0.00000001:
        if np.all(abs(residualTensor.array) < 0.0000000000000001):
            break
#        Energy = newEnergy
    print(Energy)
    print(t2Tensor.array)

In [100]:
residualTensor.array

array([[[[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00935906,  0.00673648],
         [ 0.01561124,  0.02848843]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [101]:
trueCCD.kernel()

(-0.046469771227442426,
 (array([[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]]),
  array([[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]])),
 (array([[[[ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ]],
  

In [102]:
convergeDoublesAmplitudes()

0.0
-0.024311010819762696
-0.035549815431841804
-0.04100158135429713
-0.0437073638104578
-0.045068230093333904
-0.0457570587227324
-0.04610686838249322
-0.04628483280767274
-0.04637547038416189
-0.046421665143156814
-0.04644522051463633
-0.046457235905851005
-0.04646336638372019
-0.046466494835433615
-0.04646809153055459
-0.04646890652857675
-0.046469322555975336
-0.046469534934192654
-0.04646964335551507
-0.04646969870731482
-0.04646972696638695
-0.04646974139387719
-0.046469748759825694
-0.046469752520539455
-0.046469754440598705
-0.04646975542090331
-0.04646975592140893
-0.046469756176948436
-0.046469756307417606
-0.04646975637403053
-0.046469756408040755
-0.046469756425405205
-0.046469756434270905
-0.04646975643879742
-0.04646975644110852
-0.04646975644228846
-0.04646975644289092
-0.0464697564431985
-0.04646975644335554
-0.04646975644343573
-0.04646975644347666
-0.04646975644349757
-0.046469756443508245
-0.04646975644351367
-0.04646975644351646
-0.046469756443517904
-0.046469756443

In [103]:
residualTensor.array

array([[[[-3.86416431e-18, -2.77149180e-18],
         [-2.34458720e-18,  2.77555756e-17]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-3.16769146e-18, -5.63446317e-18],
         [-5.90043151e-18,  7.15573434e-18]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-7.78762092e-18, -3.60497222e-18],
         [-2.79182059e-18,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.000

In [104]:
trueCCD

In [105]:
t2Tensor.array

array([[[[ 7.25057996e-05, -1.22582488e-03],
         [-1.22582488e-03, -1.71775864e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-8.32521512e-04, -1.22019934e-03],
         [-2.35858417e-03, -5.57834516e-02]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 7.25057996e-05, -1.22582488e-03],
         [-1.22582488e-03, -1.71775864e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.000

In [106]:
trueAmplitudeArray

array([[[[ 7.25036639e-05, -1.22582208e-03],
         [-1.22582208e-03, -1.71775938e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-8.32519695e-04, -1.22019758e-03],
         [-2.35857990e-03, -5.57834681e-02]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 7.25036639e-05, -1.22582208e-03],
         [-1.22582208e-03, -1.71775938e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.000

In [107]:
#t2Tensor.array = getDoublesMP2Amplitudes(t2Tensor)
#fsolve(testAmplitudes, t2Tensor.array.reshape((-1)))

In [108]:
t2Tensor.array

array([[[[ 7.25057996e-05, -1.22582488e-03],
         [-1.22582488e-03, -1.71775864e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-8.32521512e-04, -1.22019934e-03],
         [-2.35858417e-03, -5.57834516e-02]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 7.25057996e-05, -1.22582488e-03],
         [-1.22582488e-03, -1.71775864e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.000

In [109]:
2 * contractTensorSum(Ej) - contractTensorSum(Ek)

-0.046469756443519354

In [110]:
trueAmplitudeArray

array([[[[ 7.25036639e-05, -1.22582208e-03],
         [-1.22582208e-03, -1.71775938e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-8.32519695e-04, -1.22019758e-03],
         [-2.35857990e-03, -5.57834681e-02]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 7.25036639e-05, -1.22582208e-03],
         [-1.22582208e-03, -1.71775938e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.000

In [111]:
residualTensor.array

array([[[[-3.86416431e-18, -2.77149180e-18],
         [-2.34458720e-18,  2.77555756e-17]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-3.16769146e-18, -5.63446317e-18],
         [-5.90043151e-18,  7.15573434e-18]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-7.78762092e-18, -3.60497222e-18],
         [-2.79182059e-18,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.000

In [112]:
def amplitudesToVector(amplitudeTensor):
    amplitudeArray = amplitudeTensor.array

In [113]:
from importlib import reload

In [114]:
import texify
#reload(texify)

In [115]:

texify.texify([Ej, Ek], "CCDEnergyTerms")

In [116]:
t2Tensor.array = trueAmplitudeArray

In [117]:
2 * contractTensorSum(Ej) - contractTensorSum(Ek)

-0.046469771227442426

In [118]:
getDoublesMP2Amplitudes(t2Tensor)

array([[[[-0.00034461, -0.00127549],
         [-0.00127549, -0.08213441]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.00093956, -0.00121257],
         [-0.00281002, -0.02477347]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]]],


       [[[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.00034461, -0.00127549],
         [-0.00127549, -0.08213441]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[

In [119]:
t2Tensor.array = getDoublesMP2Amplitudes(t2Tensor)
2 * contractTensorSum(Ej) - contractTensorSum(Ek)

-0.0243110108197627

In [120]:
from pyscf import mp

In [121]:
trueMP2 = mp.MP2(mf)

In [122]:
trueMP2.kernel()

(-0.024311010805774455,
 (array([[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             2.16840434e-19],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  2.16840434e-19,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00, -2.16840434e-19,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00, -2.16840434

In [123]:
from time import time
from GeneralisedWick import *
import CC, texify
from pyscf import cc, mp
import pickle

with open("CCDEquations.pkl", 'rb') as f:
    up = pickle.Unpickler(f)
    d = up.load()
    energyEquation = d["energyEquation"]
    doublesAmplitudeEquation = d["doublesAmplitudeEquation"]
    fockTensor = d["fockTensor"]
    h2Tensor = d["h2Tensor"]
    t2Tensor = d["t2Tensor"]

In [124]:
print(doublesAmplitudeEquation)

1.0 * v_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.5 * v_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -0.5 * v_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 1.0 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + 0.5 * f_{h_{2}}^{h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{1}}
 + -1.0 * f_{h_{2}}^{h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{1}}
 + -1.0 * f_{h_{2}}^{h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{1}h_{2}}
 + 0.5 * f_{h_{2}}^{h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{1}h_{2}}
 + -1.0 * f_{h_{2}}^{h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{0}}
 + 0.5 * f_{h_{2}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{0}}
 + 0.5 * f_{h_{2}}^{h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{0}h_{2}}
 + -1.0 * f_{h_{2}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{2}}
 + -0.5 * f_{p_{1}}^{p_{3}}{t_{2}}_{p_{0}p_{3}}^{h_{1}h_{0}}
 + 1.0 * f_{p_{1}}^{p_{3}}{t_{2}}_{p_{3}p_{0}}^{h_{1}h_{0}}
 + 1.0 * f_{p_{0}}^{p_{3}}{t_{2}}_{p_{1}p_{3}}^{h_{1}h_{0}}
 + -0.5 * f_{p_{0}}^{p_{3}}{t_{2}}_{p_{3}p_{1}}^{h_{1}h_{0}}
 + 1.0 * f_{p_{1}}^{p_{3}}{t_{2}}_{p_{0}p_{3}}^{h_{0}h_{1}}
 + -0.5 * f_{p_{1}}^{p_{3}}{t_{2}}_{p_{3}p_{0}}^{h_{0}h_{1}}
 + -0.

In [125]:
print(amplitudes)

1.0 * v_{AB}^{IJ}
 + 1.0 * f_{B}^{C}{t_{2}}_{AC}^{IJ}
 + 1.0 * f_{A}^{C}{t_{2}}_{BC}^{JI}
 + -1.0 * f_{K}^{J}{t_{2}}_{AB}^{IK}
 + -1.0 * f_{K}^{I}{t_{2}}_{BA}^{JK}
 + 0.5 * v_{AB}^{CD}{t_{2}}_{CD}^{IJ}
 + 0.5 * v_{BA}^{CD}{t_{2}}_{CD}^{JI}
 + 0.5 * v_{KL}^{IJ}{t_{2}}_{AB}^{KL}
 + 0.5 * v_{KL}^{JI}{t_{2}}_{BA}^{KL}
 + 2.0 * v_{KB}^{CJ}{t_{2}}_{AC}^{IK}
 + 2.0 * v_{KA}^{CI}{t_{2}}_{BC}^{JK}
 + -1.0 * v_{KB}^{CJ}{t_{2}}_{AC}^{KI}
 + -1.0 * v_{KA}^{CI}{t_{2}}_{BC}^{KJ}
 + -1.0 * v_{KB}^{IC}{t_{2}}_{AC}^{KJ}
 + -1.0 * v_{KA}^{JC}{t_{2}}_{BC}^{KI}
 + -1.0 * v_{KB}^{JC}{t_{2}}_{AC}^{IK}
 + -1.0 * v_{KA}^{IC}{t_{2}}_{BC}^{JK}
 + 0.5 * v_{KL}^{CD}{t_{2}}_{CD}^{IJ}{t_{2}}_{AB}^{KL}
 + 0.5 * v_{KL}^{CD}{t_{2}}_{CD}^{JI}{t_{2}}_{BA}^{KL}
 + 2.0 * v_{KL}^{CD}{t_{2}}_{AC}^{IK}{t_{2}}_{DB}^{LJ}
 + 2.0 * v_{KL}^{CD}{t_{2}}_{BC}^{JK}{t_{2}}_{DA}^{LI}
 + -2.0 * v_{KL}^{CD}{t_{2}}_{AC}^{IK}{t_{2}}_{DB}^{JL}
 + -2.0 * v_{KL}^{CD}{t_{2}}_{BC}^{JK}{t_{2}}_{DA}^{IL}
 + 0.5 * v_{KL}^{CD}{t_{2}}_{CA}^{IK}{t_{2

In [126]:
contractTensorSum(amplitudes)

array([[[[-1.88105096e-03,  6.27802821e-04],
         [ 6.27802821e-04,  3.32123949e-02]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-4.61736854e-05,  6.58056514e-04],
         [ 1.47541512e-04,  1.69851044e-02]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-1.88105096e-03,  6.27802821e-04],
         [ 6.27802821e-04,  3.32123949e-02]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.000

In [127]:
print(amplitudes.collectIsomorphicTerms())

1.0 * v_{AB}^{IJ}
 + 2.0 * f_{B}^{C}{t_{2}}_{AC}^{IJ}
 + -2.0 * f_{K}^{J}{t_{2}}_{AB}^{IK}
 + 1.0 * v_{AB}^{CD}{t_{2}}_{CD}^{IJ}
 + 1.0 * v_{KL}^{IJ}{t_{2}}_{AB}^{KL}
 + 4.0 * v_{KB}^{CJ}{t_{2}}_{AC}^{IK}
 + -2.0 * v_{KB}^{CJ}{t_{2}}_{AC}^{KI}
 + -2.0 * v_{KB}^{IC}{t_{2}}_{AC}^{KJ}
 + -2.0 * v_{KB}^{JC}{t_{2}}_{AC}^{IK}
 + 1.0 * v_{KL}^{CD}{t_{2}}_{CD}^{IJ}{t_{2}}_{AB}^{KL}
 + 4.0 * v_{KL}^{CD}{t_{2}}_{AC}^{IK}{t_{2}}_{DB}^{LJ}
 + -4.0 * v_{KL}^{CD}{t_{2}}_{AC}^{IK}{t_{2}}_{DB}^{JL}
 + 1.0 * v_{KL}^{CD}{t_{2}}_{CA}^{IK}{t_{2}}_{BD}^{LJ}
 + -2.0 * v_{KL}^{CD}{t_{2}}_{AD}^{IK}{t_{2}}_{CB}^{LJ}
 + 2.0 * v_{KL}^{CD}{t_{2}}_{AD}^{KI}{t_{2}}_{CB}^{LJ}
 + 1.0 * v_{KL}^{CD}{t_{2}}_{CB}^{IL}{t_{2}}_{AD}^{KJ}
 + -4.0 * v_{KL}^{CD}{t_{2}}_{CD}^{KI}{t_{2}}_{AB}^{LJ}
 + 2.0 * v_{KL}^{CD}{t_{2}}_{CD}^{IK}{t_{2}}_{AB}^{LJ}
 + -4.0 * v_{KL}^{CD}{t_{2}}_{CA}^{KL}{t_{2}}_{DB}^{IJ}
 + 2.0 * v_{KL}^{CD}{t_{2}}_{AC}^{KL}{t_{2}}_{DB}^{IJ}


In [128]:
sum([getContractedArray(summand)[0] for summand in amplitudes.summandList])

array([[[[-0.00188105, -0.00549474],
         [ 0.00675034,  0.03321239]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00071237, -0.01759685],
         [ 0.01459246,  0.01563282]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00188105, -0.00549474],
         [ 0.00675034,  0.03321239]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [129]:
sum([getContractedArray(summand)[0] for summand in amplitudes.collectIsomorphicTerms().summandList])

array([[[[-0.00188105, -0.00549474],
         [ 0.00675034,  0.03321239]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00071237, -0.01759685],
         [ 0.01459246,  0.01563282]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00188105, -0.00549474],
         [ 0.00675034,  0.03321239]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

Residual from incomplete contractions

In [130]:

normalOrderedHamiltonian = sum(fockTensor.diagrams) + (1. / 2.) * sum(h2Tensor.diagrams)
BCH = CC.BCHSimilarityTransform(normalOrderedHamiltonian, 0.5 * t2Tensor, 2)

In [131]:
CCDEnergy = evaluateWickFree(BCH, True)

In [132]:
print(CCDEnergy)

0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.125 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -0.125 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{1}}


In [133]:
CCDDoubleResidual = evaluateWickFree(BCH, True, (["p", "p"], ["h", "h"]))

In [134]:
print(CCDDoubleResidual)

0.125 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + 0.125 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + 0.125 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + 0.125 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -0.125 * f_{h_{0}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{3}}
 + 0.125 * f_{h_{0}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{0}}
 + 0.125 * f_{h_{0}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{0}}
 + -0.125 * f_{h_{0}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{3}}
 + -0.125 * f_{h_{0}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{3}}
 + 0.125 * f_{h_{0}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{0}}
 + -0.125 * f_{h_{0}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{3}}
 + 0.125 * f_{h_{0}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{0}}
 + -0.125 * f_{p_{0}}^{p_{1}}{t_{2}}_{p_{2}p_{1}}^{h_{0}h_{1}}
 + 0.125 * f_{p_{0}}^{p_{1}}{t_{2}}_{p_{1}p_{3}}^{h_{0}h_{1}}
 + -0.125 * f_{p_{0}}^{p_{1}}{t_{2}}_{p_{2}p_{1}}^{h_{0}h_{1}}
 + 0.125 * f_{p_{0}}^{p_{1}}{t_{2}}_{p_{1}p_{3}}^{h_{0}h_{1}}
 + 0.125 * f_{p_{0}}^{p_{1}}{t_{2}}_{p_{1}p_{3}}^{h_{0}h_{1}}
 + -0.125 * f_{p_{0}}^{p_{1}}{t_{2}

In [135]:
h2Tensor.array = eri.reshape((Norbs, Norbs, Norbs, Norbs)).swapaxes(2,3).swapaxes(1,2)

fock = h1
for p in range(Norbs):
    for q in range(Norbs):
        fock[p,q] += sum([2 * h2Tensor.array[p,i,q,i] - h2Tensor.array[p,i,i,q] for i in range(Nocc)])
fockTensor.array = fock

fockTensor.assignDiagramArrays(vacuum)
h2Tensor.assignDiagramArrays(vacuum)
t2Tensor.getShape(vacuum)

In [136]:
sum([getContractedArray(summand)[0] for summand in CCDDoubleResidual.summandList])

array([[[[ 0.00165043,  0.00329909],
         [ 0.00329909,  0.0315175 ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00467953,  0.00336824],
         [ 0.00780562,  0.01424421]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00165043,  0.00329909],
         [ 0.00329909,  0.0315175 ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [137]:
print(contractTensorSum(CCDDoubleResidual))

[[[[ 0.00165043  0.00329909]
   [ 0.00329909  0.0315175 ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00467953  0.00336824]
   [ 0.00780562  0.01424421]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]]


 [[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00165043  0.00329909]
   [ 0.00329909  0.0315175 ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00467953  0.00336824]
   [ 0.00780562  0.01424421]]]


 [[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00165043  0.00329909]
   [ 0.

In [138]:
CC.convergeDoublesAmplitudes(t2Tensor, CCDEnergy, CCDDoubleResidual, fockTensor)

0.0
-0.003034919472771903
-0.00576293131691832
-0.00815085434799128
-0.010157783534077862
-0.011735739884216844
-0.012830277361093087
-0.013380869252211776
-0.013320818114132712
-0.012576349198801122
-0.0110644501785967
-0.008688890817605918
-0.005333655908575027
-0.0008526818224625922
0.004945826042661338
0.012323332832837028
0.021648068098216112
0.03345406240336871
0.04853852058356299
0.06812946719997323
0.094189144973133
0.12999594430461442
0.18134021168903014
0.25919647311104776
0.3863558917783493
0.6162803603549438
1.0974180931073123
2.3570830886855534
7.153634587486459
43.79893973355077
1225.628192398865
876504.5711964994
465252078889.16516
1.4086562629608455e+23
1.3893486027378055e+46
1.4415563141054986e+92
1.6331359918582093e+184


KeyboardInterrupt: 

In [139]:
BCHconst = normalOrderedHamiltonian
BCHlin = CC.commutator(normalOrderedHamiltonian, 0.5 * t2Tensor)
BCHquad = 0.5 * CC.commutator(CC.commutator(normalOrderedHamiltonian, 0.5 * t2Tensor), 0.5 * t2Tensor)

In [140]:
CCDDoubleconst = evaluateWickFree(BCHconst, True, (["p", "p"], ["h", "h"]))
CCDDoublelin = evaluateWickFree(BCHlin, True, (["p", "p"], ["h", "h"]))
CCDDoublequad = evaluateWickFree(BCHquad, True, (["p", "p"], ["h", "h"]))

In [141]:
print(CCDDoublequad)

0.0625 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{2}p_{3}}^{h_{2}h_{3}}{t_{2}}_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.03125 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{2}p_{3}}^{h_{2}h_{3}}{t_{2}}_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -0.03125 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{2}p_{3}}^{h_{2}h_{3}}{t_{2}}_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 0.0625 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{2}p_{3}}^{h_{2}h_{3}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -0.03125 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{2}p_{3}}^{h_{0}h_{3}}{t_{2}}_{p_{1}p_{0}}^{h_{1}h_{5}}
 + 0.015625 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{2}p_{3}}^{h_{0}h_{3}}{t_{2}}_{p_{0}p_{1}}^{h_{1}h_{5}}
 + 0.015625 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{2}p_{3}}^{h_{1}h_{3}}{t_{2}}_{p_{1}p_{0}}^{h_{0}h_{5}}
 + -0.03125 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{2}p_{3}}^{h_{1}h_{3}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{5}}
 + 0.015625 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{2}p_{3}}^{h_{0}h_{3}}{t_{2}}_{p_{1}p_{0}}^{h_{4}h_{1}}
 + -0.03125 * v_{h_{0}h_{1}}^{p_{0}